# **Tính chỉ số Faithfulness Index (FI)**

### **Mục tiêu:**

Thực hiện đo lường chỉ số Faithfulness Index.

Mục tiêu là kiểm chứng xem các mẫu train top-k có thực sự ảnh hưởng mạnh hơn các mẫu ngẫu nhiên (random) hay không?



### **Mã nguồn**

Thực hiện 3 bước chính sau:

**1. Đảm bảo rằng Help/Harm score sẽ được so sánh nếu cả Top-k và Random của 1 mẫu test đều tồn tại.**

- Hàm `generate_unique_key` tạo ID cho mỗi mẫu test: `corpus_id + sentence index + span_id`

- Nếu ở tập top-k và tập random đều tìm được ID này thì mới tiến hành tính toán.

- Nếu không thì mẫu đó sẽ bị bỏ qua.

**2. Trích xuất điểm Help/Harm.**

- Hàm `extract_valid_scores` lấy danh sách điểm `help_harm_score` từ file json (top-k và random).

**3. Tính FI.**

Với mỗi mẫu test $x_{ts}$, chỉ số FI được tính như sau:

$$
FI(x_{ts}) = \frac{1}{k} \sum_{i=1}^{k} \frac{\text{count}(|S_{random}| < |S_{top\_k\_i}|)}{M}
$$

**Trong đó:**

* $|S_{top\_k\_i}|$: Độ lớn tuyệt đối của điểm Help/Harm của láng giềng thứ $i$ trong Top-k.
* $|S_{random}|$: Độ lớn tuyệt đối của điểm Help/Harm của các mẫu trong tập Random.
* $M$: Tổng số mẫu Random (Baseline).

Code sẽ chạy qua 12 layer (lược bỏ layer 0 vì nó chỉ liên quan đến embedding).
- Tính trung bình cộng FI của tất cả các mẫu test hợp lệ trong layer đó.
- Tính trung bình cộng FI của toàn bộ 12 layer.


Nếu trung bình FI của 12 layer:
- **FI > 0.5:** Các mẫu train tìm được bằng phương pháp đề xuất có ảnh hưởng mạnh hơn so với các mẫu ngẫu nhiên.
- **FI <= 0.5**: Các mẫu train không thật sự ảnh hưởng mạnh so với các mẫu ngẫu nhiên.

In [ ]:
# Import thư viện:
import json
import os
import numpy as np
import glob
from tqdm import tqdm

In [ ]:
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Thư mục chứa kết quả Top-5 (Top-k)
TOP_K_DIR = os.path.join(drive_base_path, 'search_results/v2_neutralize_with_cosine_similarity/layer_wise_results_with_help_harm_scores')

# Thư mục chứa kết quả Random 100 (Baseline)
RANDOM_BASE_DIR = os.path.join(drive_base_path, 'search_results/v2_neutralize_with_cosine_similarity/layer_results_with_help_harm_scores_stratified_from_top500')

NUM_LAYERS = 12

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def generate_unique_key(item):
    """
    Tạo ID duy nhất: corpus_id | sentence_idx | span_id
    Để ghép đôi chính xác mẫu test bên file Top-5 và file Random.
    """
    try:
        info = item.get('query_info', {})
        c_id = info.get('corpus_id', 'unknown')
        s_idx = info.get('sentence_idx', 'unknown')
        sp_id = info.get('span_id', 'unknown')
        return f"{c_id}|{s_idx}|{sp_id}"
    except Exception:
        return None

def extract_valid_scores(item, key_name):
    """
    Lấy danh sách điểm Help/Harm.
    QUAN TRỌNG: Lọc bỏ các giá trị None (do lỗi thiếu vector).
    """
    scores = []
    data_list = item.get(key_name, [])

    if isinstance(data_list, list):
        for entry in data_list:
            # Chỉ lấy nếu có key help_harm_score VÀ giá trị không phải None
            if 'help_harm_score' in entry and entry['help_harm_score'] is not None:
                scores.append(float(entry['help_harm_score']))
    return scores

def get_random_data_map(layer_idx, random_base_dir):
    """
    Load toàn bộ dữ liệu Random của layer đó vào Dictionary.
    Output: { "ID_unique": [list_diem_so_random] }
    """
    mapping = {}
    # Quét tất cả các folder con (batch 0-500, 500-1000...)
    search_pattern = os.path.join(random_base_dir, "*", f"search_results_layer_{layer_idx}.json")
    file_list = glob.glob(search_pattern)

    print(f"  -> Tìm thấy {len(file_list)} file dữ liệu Random cho Layer {layer_idx}")

    for file_path in file_list:
        try:
            data_list = load_json(file_path)
            if isinstance(data_list, list):
                for item in data_list:
                    key = generate_unique_key(item)
                    # File Random key là: 'random_baseline_neighbors'
                    scores = extract_valid_scores(item, 'random_baseline_neighbors')
                    if key and scores:
                        mapping[key] = scores
        except Exception as e:
            print(f"Lỗi đọc file {os.path.basename(file_path)}: {e}")

    return mapping

def calculate_fi_only(layer_idx):
    print(f"\n--- Đang xử lý Layer {layer_idx} ---")

    # Load file Top-5
    top_k_path = os.path.join(TOP_K_DIR, f"search_results_layer_{layer_idx}.json")
    if not os.path.exists(top_k_path):
        print(f"Không tìm thấy file Top-K: {top_k_path}")
        return None

    top_k_data = load_json(top_k_path)

    # Load dữ liệu Random để làm baseline
    random_map = get_random_data_map(layer_idx, RANDOM_BASE_DIR)

    if not random_map:
        print("Không load được dữ liệu Random nào. Bỏ qua layer này.")
        return None

    total_fi = 0
    valid_sample_count = 0

    # Tính FI cho từng mẫu
    for item in top_k_data:
        key = generate_unique_key(item)
        if key not in random_map:
            continue

        # Lấy điểm số Top-5 và Random tương ứng
        top_k_scores = extract_valid_scores(item, 'neighbors')
        random_scores = random_map[key]

        # Kiểm tra dữ liệu rỗng
        if not top_k_scores or not random_scores:
            continue

        # FI = (1/k) * Sum( số lượng random < top_k_i ) / total_random

        k = len(top_k_scores)
        abs_random = np.abs(random_scores)
        sum_ratios = 0

        for score in top_k_scores:
            abs_top = abs(score)
            count_smaller = np.sum(abs_random < abs_top)
            ratio = count_smaller / len(random_scores)
            sum_ratios += ratio

        sample_fi = sum_ratios / k

        total_fi += sample_fi
        valid_sample_count += 1

    # Tính trung bình FI cho cả Layer
    final_fi = total_fi / valid_sample_count if valid_sample_count > 0 else 0

    return {
        "layer": layer_idx,
        "FI": final_fi,
        "processed_samples": valid_sample_count
    }

if __name__ == "__main__":
    results = []

    for i in range(1, NUM_LAYERS + 1):
        res = calculate_fi_only(i)
        if res:
            results.append(res)
            print(f"Layer {i}: FI = {res['FI']:.4f} (Dựa trên {res['processed_samples']} mẫu khớp)")

    if results:
        avg_fi = np.mean([r['FI'] for r in results])
        print(f"KẾT QUẢ CUỐI CÙNG (Average FI all layers): {avg_fi:.4f}\n")

        # Nhận xét nhanh dựa trên tài liệu
        if avg_fi > 0.5:
            print("=> Kết luận: Top-k neighbors có ảnh hưởng thực sự lớn hơn Random (FI > 0.5).")
        else:
            print("=> Kết luận: Top-k neighbors không tốt hơn Random (FI <= 0.5).")


--- Đang xử lý Layer 1 ---
  -> Tìm thấy 5 file dữ liệu Random cho Layer 1
Layer 1: FI = 0.5202 (Dựa trên 2071 mẫu khớp)

--- Đang xử lý Layer 2 ---
  -> Tìm thấy 5 file dữ liệu Random cho Layer 2
Layer 2: FI = 0.6763 (Dựa trên 2071 mẫu khớp)

--- Đang xử lý Layer 3 ---
  -> Tìm thấy 5 file dữ liệu Random cho Layer 3
Layer 3: FI = 0.7185 (Dựa trên 2071 mẫu khớp)

--- Đang xử lý Layer 4 ---
  -> Tìm thấy 5 file dữ liệu Random cho Layer 4
Layer 4: FI = 0.8045 (Dựa trên 2071 mẫu khớp)

--- Đang xử lý Layer 5 ---
  -> Tìm thấy 5 file dữ liệu Random cho Layer 5
Layer 5: FI = 0.8166 (Dựa trên 2071 mẫu khớp)

--- Đang xử lý Layer 6 ---
  -> Tìm thấy 5 file dữ liệu Random cho Layer 6
Layer 6: FI = 0.8457 (Dựa trên 2071 mẫu khớp)

--- Đang xử lý Layer 7 ---
  -> Tìm thấy 5 file dữ liệu Random cho Layer 7
Layer 7: FI = 0.8378 (Dựa trên 2071 mẫu khớp)

--- Đang xử lý Layer 8 ---
  -> Tìm thấy 5 file dữ liệu Random cho Layer 8
Layer 8: FI = 0.8865 (Dựa trên 2071 mẫu khớp)

--- Đang xử lý Layer 9 